<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/apunte_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apunte de Spark

## MapReduce paradigm

Papers relevante: https://www.sciencedirect.com/science/article/pii/S0167642307001281?via%3Dihub

http://static.googleusercontent.com/media/research.google.com/es/us/archive/mapreduce-osdi04.pdf

**MapReduce** es un paradigma de procesamiento de grandes volúmenes de datos basado en la composición de tres funciones:
- Una función de **map**, que aplica una función dependiente del dominio a un conjunto de pares clave-valor procesándolo a otro conjunto de pares clave-valor
- Una función de **groupby** o **agrupación** que agrupa los pares según grupos de llaves
- Una función **reduce** que reduce el conjunto de pares atención a ningún o un output determinado.

Un ejemplo de aplicación del paradigma MapReduce es en buscar la cantidad de ocurrencias de cada palabra que aparezca al menos una vez en un conjunto de documentos. Para conseguir ello, primero aplicaríamos una función **map** que, por cada documento, contaría la cantidad de veces que aparece cada palabra del documento. Luego, en este caso la función **groupby** es una identidad, pues no necesitamos agrupar los documentos. Finalmente, nuestro **reduce** es una función cuyos inputs son pares documento-conteos de palabras, y su output es un conteo de palabras para cada palabra que estuviese en los conteos de los inputs.

Las ventajas del paradigma MapReduce es que es fácilmente paralelizable, y unifica buena parte de las transformaciones/consultas a bases de datos masivas que podemos encontrar.

Para paralelizar efectivamente las operaciones, se suele necesitar un **cluster** de computadoras, que consiste en un conjunto de computadoras que trabajan juntas y pueden verse como un sistema único. Cada computadora del clúster se conoce como **nodo**.

Además, se suelen almacenar los datos de forma **distribuida**, es decir, los datos se encuentran almacenados en distintos clusters. 

Cuando tenemos almacenamiento distribuido, a las etapas de map y reduce tenemos que añadir una etapa de **shuffle & sort** debido que que combinar directamente los datos de todos los nodos distribuidos es muy caro. En esta etapa vamos aplicando las funciones de a pares, y progresivamente vamos juntando los resultados de cada par de nodos para llegar al resultado final. 

## Apache Spark

Paper relevante: https://amplab.cs.berkeley.edu/wp-content/uploads/2011/06/Spark-Cluster-Computing-with-Working-Sets.pdf

**Spark** es un framework de procesamiento de datos masivos para aplicaciones que reusan dicho set de datos a través de múltiples operaciones paralelizadas. En particular, Spark se utiliza para dos tipos de aplicaciones: trabajos **iterativos**, como la aplicación de una función de una forma sucesiva en aplicaciones de machine learning, y **análisis interactivo de datos**.

La abstracción principal que caracteriza Spark es el **RDD** o **resilient distributed dataset**, que representa una colección read-only de objetos particionada en un conjunto de máquinas que puede ser reconstruida si una parte se pierde. 

Para utilizar Spark, los desarrolladores escriben un programa **driver** que implementa el flujo de control de la aplicación a alto nivel y lanza varias operaciones en paralelo. Utilizando RDD's y **operaciones paralelas** en dichos datasets como las abstracciones principales se logra dicho paradigma de programación paralela. También soporta **variables compartidas** (*shared variables*) que pueden utilizarse en funciones aplicadas en el cluster entero. 

### RDD

Un **resilient distributed dataset** o RDD es una colección read-only de objetos partidos en diferentes máquinas que puede reconstruirse si se pierde una parte. Spark permite construir RDD's de diferentes maneras:
- Desde un archivo de un filesystem
- "paralelizando" una colección (array) de Scala en el driver
- Transformando un RDD existente
- Cambiando la *persistencia* de un RDD existente

### Operaciones paralelas (Parallel operations)

Spark soporta varias operaciones paralelas que pueden ser aplicadas a RDD's:
- *reduce*: Combina elementos del dataset utilizando una función de agregación para producir un resultado al final del driver.
- *collect*: Manda un subconjunto de los elementos al driver.
- *foreach*: Aplica una función definida por el usuario a cada elemento en el driver. 

### Variables compartidas

Tras aplicar funciones como *map*, *filter*, *reduce*, etc pasando closures a Spark en un determinado **worker node**, Spark también soporta dos patrones de uso para generalizar el resultado a todo el cluster:
- **Broadcast variables**, aplicada cuando una pieza read-only muy grande es utilizada en múltiples operaciones paralelas y lo más óptimo es distribuir los resultados a los workers una vez finalizadas las operaciones.
- **Accumulators**, que sirven para implementar contadores como en MapReduce, y solo pueden realizar operaciones de adición.